# TSA Chapter 3: ARIMA Forecasting with Confidence Intervals

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_case_forecast/TSA_ch3_case_forecast.ipynb)

This notebook demonstrates:
- 8-quarter ahead ARIMA forecast with 95% confidence bands for US Real GDP.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader -q

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport pandas_datareader as pdrfrom statsmodels.tsa.arima.model import ARIMA

In [ ]:
# Chart style settings - Nature journal qualityplt.rcParams['figure.facecolor'] = 'none'plt.rcParams['axes.facecolor'] = 'none'plt.rcParams['savefig.facecolor'] = 'none'plt.rcParams['axes.grid'] = Falseplt.rcParams['font.family'] = 'sans-serif'plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']plt.rcParams['font.size'] = 8plt.rcParams['axes.labelsize'] = 9plt.rcParams['axes.titlesize'] = 10plt.rcParams['xtick.labelsize'] = 8plt.rcParams['ytick.labelsize'] = 8plt.rcParams['legend.fontsize'] = 8plt.rcParams['legend.facecolor'] = 'none'plt.rcParams['legend.framealpha'] = 0plt.rcParams['axes.spines.top'] = Falseplt.rcParams['axes.spines.right'] = Falseplt.rcParams['axes.linewidth'] = 0.5plt.rcParams['lines.linewidth'] = 0.75BLUE, RED = '#1A3A6E', '#DC3545'# Get data# Fit model# Forecast# Create future dates# Plot last 40 observations# Forecast# Visual separator between historical and forecastplt.tight_layout()plt.savefig('ch3_case_forecast.pdf', dpi=300, bbox_inches='tight')plt.show()

In [ ]:
gdp = pdr.get_data_fred('GDPC1', start='1960-01-01', end='2024-09-30')
gdp_data = gdp['GDPC1'].dropna()
log_gdp = np.log(gdp_data)
model = ARIMA(log_gdp, order=(1, 1, 1))
fit = model.fit()
forecast_steps = 8
forecast = fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()
last_date = gdp_data.index[-1]
future_dates = pd.date_range(start=last_date, periods=forecast_steps + 1, freq='Q')[1:]
fig, ax = plt.subplots(figsize=(14, 6))
plot_data = log_gdp.iloc[-40:]
ax.plot(plot_data.index, plot_data.values, color=BLUE, linewidth=2, label='Observed')
ax.plot(future_dates, forecast_mean.values, color=RED, linewidth=2,
        linestyle='--', label='Forecast')
ax.fill_between(future_dates, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1],
                color=RED, alpha=0.2, label='95% CI')
split_point = plot_data.index[-1]
ax.axvline(x=split_point, color='black', linestyle='--', linewidth=1.5, alpha=0.7)
y_pos = ax.get_ylim()[1] - 0.02 * (ax.get_ylim()[1] - ax.get_ylim()[0])
ax.text(split_point, y_pos, '  Forecast ', fontsize=9, ha='left', va='top',
        color='black', fontweight='bold', alpha=0.8)
ax.set_title('US Real GDP: ARIMA Forecast', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Log(GDP)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
ax.text(0.02, 0.98, 'Model: ARIMA(1,1,1)\nSource: FRED (GDPC1)',
        transform=ax.transAxes, fontsize=9, verticalalignment='top',
        style='italic', color='gray')